In [ ]:
# import necessary packages

from load_data import *
from embeddings_utils import *
from model_training import *
from tensorflow.keras.models import load_model


In [ ]:
# load data and split into train and validation sets
dataload = LoadData()
df = dataload.load_data("path_to_data")
X_train, X_val, y_train, y_val = dataload.split_data(df)


In [ ]:
# prepare data by spliting, padding, and truncating sentences
X_train_sentences , X_val_sentences = dataload.sentence_data_prep(X_train, X_val)


In [ ]:
# load pre-trained word embeddings
X_train_embeddings = np.load('path/to/X_train_embeddings.npy')
X_val_embeddings = np.load('path/to/X_val_embeddings.npy')

In [ ]:
# load pre-trained sentence embeddings
X_train_sentence_embeddings = load_embeddings("path/to/file",'X_train_sentence')
X_val_sentence_embeddings = load_embeddings("path/to/file",'X_val_sentence')


In [ ]:
# Remove the extra element if present
if X_train_sentence_embeddings[-1].shape != (len(X_train), 5, 512):
    X_train_sentence_embeddings.pop()

if X_val_sentence_embeddings[-1].shape != (len(X_val), 5, 512):
    X_val_sentence_embeddings.pop()


In [ ]:
# print the shapes of the data to make sure everything is in order
print("X_train_embeddings shape:", X_train_embeddings.shape)
print("X_val_embeddings shape:", X_val_embeddings.shape)
print("X_train_sentence_embeddings shapes:", [x.shape for x in X_train_sentence_embeddings])
print("X_val_sentence_embeddings shapes:", [x.shape for x in X_val_sentence_embeddings])
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


In [ ]:
# initialize the model
model_training = ModelTraining()
model = model_training.model_build()


In [ ]:
# compile the model 
strategy = tf.distribute.MirroredStrategy()
model = model_training.model_compile(model, strategy)

In [ ]:
# fit the model with the data
history, model = model_training.model_fit(model, X_train_embeddings, X_train_sentence_embeddings, y_train, X_val_embeddings, X_val_sentence_embeddings, y_val, strategy)

In [ ]:
# save the model
model.save('path/to/model.h5')

In [ ]:
# load the model after training and saving
model = load_model('path/to/model.h5')


In [ ]:
# evaluate the model on the validation set
model_training.model_evaluate(model, X_val_embeddings, X_val_sentence_embeddings, y_val, strategy)